In [50]:
import pandas as pd
alertl = pd.read_csv('AlertLevel.csv')
rvbsm = pd.read_csv('RvBsm.csv')
summary = pd.read_csv('Summary.csv')
eventappid = pd.read_csv('EventAppID.csv')
host = pd.read_csv('Host.csv')
rvzone = pd.read_csv('RvZone.csv')
evtwarn = pd.read_csv('EvtWarn.csv')
vehclass = pd.read_csv('RvBasicVehClass.csv')
spat = pd.read_csv('Spat.csv')

In [16]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

In [66]:
summary.groupby(['Device','Trip']).size().reset_index().rename(columns={0:'count'}).head()

,Device,Trip,count
0,2004,4,1
1,2004,8,1
2,2004,11,1
3,2004,12,1
4,2004,13,1


In [15]:
# unique device and trip : loop over summary 
device = 2004
trip = 12

In [16]:
summary[(summary['Device'] == device) & (summary['Trip'] == trip)]

,Device,Trip,StartTime,Endtime,UTCTime,TripStart,TodTripStart
3,2004,12,20450,136050,1539036877954,43381.926828,43381.718495


In [56]:
# filter data for the particular device and trip ids from other tables
host_data = host[(host['Device'] == device) & (host['Trip'] == trip)].reset_index().drop(columns=['index'])
rvbsm_data = rvbsm[(rvbsm['Device'] == device) & (rvbsm['Trip'] == trip)].reset_index().drop(columns=['index'])
evt_data = evtwarn[(evtwarn['Device'] == device) & (evtwarn['Trip'] == trip)].reset_index().drop(columns=['index'])
spat_data = spat[(spat['Device'] == device) & (spat['Trip'] == trip)].reset_index().drop(columns=['index'])

In [58]:
# add unique identifer to join tables on: format: device_trip_time
host_data['merge_id'] = host_data['Device'].astype(str) +'_'+ host_data['Trip'].astype(str)  +'_'+ host_data['Time'].astype(str)
rvbsm_data['merge_id'] = rvbsm_data['Device'].astype(str) +'_'+ rvbsm_data['Trip'].astype(str)  +'_'+ rvbsm_data['Time'].astype(str)
evt_data['merge_id'] = evt_data['Device'].astype(str) +'_'+ evt_data['Trip'].astype(str)  +'_'+ evt_data['Time'].astype(str)
spat_data['merge_id'] = spat_data['Device'].astype(str) +'_'+ spat_data['Trip'].astype(str)  +'_'+ spat_data['Time'].astype(str)

In [60]:
# update index to unique identifiers for the tables
host_data = host_data.set_index('merge_id')
rvbsm_data = rvbsm_data.set_index('merge_id')
evt_data = evt_data.set_index('merge_id')
spat_data = spat_data.set_index('merge_id')

In [62]:
# join the tables
ht_ = host_data.join(rvbsm_data,lsuffix='_host',rsuffix='_rvbsm').join(evt_data,rsuffix='_evtwarn').join(spat_data,rsuffix='_spat')

In [1]:
# save it as a trip (probably)
ht_.to_csv('trip0.csv')

NameError: name 'ht_' is not defined

In [72]:
pd.DataFrame(ht_.columns)

,0
0,Device_host
1,Trip_host
2,Time_host
3,NativeFlag_host
4,LocalTimeMS_host
5,Latitude_host
6,Longitude_host
7,Elevation_host
8,Heading_host
9,GpsSpeed_host


In [78]:
ht_.dropna(thresh=50)

,Device_host,Trip_host,Time_host,NativeFlag_host,LocalTimeMS_host,Latitude_host,Longitude_host,Elevation_host,Heading_host,GpsSpeed_host,GpsFixType,GpsSemiMajAxis,GpsSemiMinAxis,GpsOrientSemiMaj,BrakeStatus_host,Speed,YawRate_host,LongAccel_host,ThrottlePosPct,SteerAngle,TurnSignal,Headlamp,Wiper,TransState,StabilityControlStatus,ABSStatus,TracControlStatus,ClosestIntersectId,Distance,Device_rvbsm,Trip_rvbsm,Time_rvbsm,RvDevice,NativeFlag_rvbsm,LocalTimeMS_rvbsm,BsmPsId,BasicVehClass,RvRandomId,Latitude_rvbsm,Longitude_rvbsm,Elevation_rvbsm,Heading_rvbsm,GpsSpeed_rvbsm,BrakeStatus_rvbsm,YawRate_rvbsm,LongAccel_rvbsm,Device,Trip,Time,RvDevice_evtwarn,NativeFlag,LocalTimeMS,PsId,RvBasicVehClass,RvRandomId_evtwarn,AlertLevel,EventAppId,RvClass,RvLatitude,RvLongitude,RvHeading,RvElevation,RvSpeed,RvBrakeStatus,RvYawRate,RvLongitudinalAccel,RvTurnSignal,RvEventFlags,RvRange,RvRangeRate,RvLongOffset,RvLatOffset,HvLatitude,HvLongitude,HvHeading,HvElevation,HvSpeed,HvBrakeStatus,HvYawRate,HvLongitudinalAccel,HvTurnSignal,Device_spat,Trip_spat,Time_spat,NativeFlag_spat,LocalTimeMS_spat,IntersectId,LaneId,MovementPhase,SignalGroupId,MinEndTime
merge_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004_12_21950,2004,12,21950,0,80297394,40.279086,-83.531299,295.42001,177.78999,19.717403,2,0.99,0.62,139.0,1,19.166666,2.19,-2.6638,4,5.59375,2,0,0,2,0,0,0,2103,2100.3889,2004.0,12.0,21950.0,2494.0,0.0,80297394.0,1.0,0.0,1368.0,40.280633,-83.531419,294.79999,178.12500,23.180000,0.0,-0.97,-0.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004.0,12.0,21950.0,0.0,80297394.0,2103.0,11.0,4.0,4.0,11054.0
2004_12_22000,2004,12,22000,0,80297894,40.279001,-83.531296,295.51999,178.62000,18.439362,2,0.98,0.62,140.0,1,18.174999,1.70,-2.5280,4,4.00000,2,0,0,2,0,0,0,2103,2109.4763,2004.0,12.0,22000.0,2494.0,0.0,80297894.0,1.0,0.0,1368.0,40.280527,-83.531412,294.89999,177.25000,23.080000,0.0,-0.48,-0.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004.0,12.0,22000.0,0.0,80297894.0,2103.0,11.0,4.0,4.0,11061.0
2004_12_22050,2004,12,22050,0,80298394,40.278937,-83.531295,295.51999,179.33000,17.470680,2,0.97,0.62,141.0,1,17.147223,0.97,-2.5280,4,1.50000,2,0,0,2,0,0,0,2103,2118.0500,2004.0,12.0,22050.0,2494.0,0.0,80298394.0,1.0,0.0,1368.0,40.280422,-83.531403,295.10001,176.21249,22.879999,0.0,-0.24,-0.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004.0,12.0,22050.0,0.0,80298394.0,2103.0,11.0,4.0,4.0,11065.0
2004_12_22100,2004,12,22100,0,80298894,40.278847,-83.531295,295.62000,179.82001,16.185959,2,0.96,0.62,142.0,1,15.977777,0.48,-1.8102,4,-1.37500,2,0,0,2,0,0,0,2103,2126.0388,2004.0,12.0,22100.0,2494.0,0.0,80298894.0,1.0,0.0,1368.0,40.280318,-83.531395,295.10001,176.30000,22.740000,0.0,-0.48,-0.18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004.0,12.0,22100.0,0.0,80298894.0,2103.0,11.0,4.0,4.0,11069.0
2004_12_22150,2004,12,22150,0,80299394,40.278777,-83.531295,295.62000,180.14000,15.300014,2,0.95,0.62,143.0,1,14.883334,-0.24,-1.4610,4,-4.59375,2,0,0,2,0,0,0,2103,2133.4805,2004.0,12.0,22150.0,2494.0,0.0,80299394.0,1.0,0.0,1368.0,40.280215,-83.531386,295.29999,176.18750,22.600000,0.0,0.00,-0.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004.0,12.0,22150.0,0.0,80299394.0,2103.0,11.0,4.0,4.0,11074.0
2004_12_22200,2004,12,22200,0,80299894,40.278723,-83.531295,295.72000,179.75999,14.702361,2,0.94,0.62,143.0,1,14.416667,-0.48,-1.1894,4,-7.59375,2,0,0,2,0,0,0,2103,2140.6890,2004.0,12.0,22200.0,2494.0,0.0,80299894.0,1.0,0.0,1368.0,40.280114,-83.531377

In [ ]:
ht_.drop(columns=['LocalTimeMS_spat','NativeFlag_spat','Time_spat','Trip_spat','Device_spat','RvRandomId_evtwarn','RvDevice_evtwarn'])

In [51]:
# Ignore things above (eda)
# filter event with appid = 2
evt_ = evtwarn[(evtwarn['EventAppId'] == 2)]

In [52]:
# get unique trip and device. ignore consecutive events with interval less than 2.5s
ignore_interval = 2.5 * 100
e_  = pd.DataFrame(columns=evt_.columns)
i_ = 0
for i, row in evt_.iterrows():
    if i_==0:
        e_.loc[i_] = row
        i_+=1
    else:
        if ((i-1 in evt_.index) and (evt_.loc[i]['Trip']==evt_.loc[i-1]['Trip']) \
        and (evt_.loc[i]['Device']==evt_.loc[i-1]['Device']) and (evt_.loc[i]['Time']-evt_.loc[i-1]['Time']<=ignore_interval)):
            pass
        else:
            e_.loc[i_] = row
            i_+=1
            

In [56]:
# e_.loc[0]['Time']-list(host[(host['Device']==e_.loc[0]['Device']) & (host['Trip']==e_.loc[0]['Trip']) & (host['Time']>e_.loc[0]['Time']) & (host['Time']<(e_.loc[0]['Time']+6000)) & (host['BrakeStatus']==1)]['Time'])[0]

In [53]:
# new table with unique warings
e_.head()

,Device,Trip,Time,RvDevice,NativeFlag,LocalTimeMS,PsId,RvBasicVehClass,RvRandomId,AlertLevel,EventAppId,RvClass,RvLatitude,RvLongitude,RvHeading,RvElevation,RvSpeed,RvBrakeStatus,RvYawRate,RvLongitudinalAccel,RvTurnSignal,RvEventFlags,RvRange,RvRangeRate,RvLongOffset,RvLatOffset,HvLatitude,HvLongitude,HvHeading,HvElevation,HvSpeed,HvBrakeStatus,HvYawRate,HvLongitudinalAccel,HvTurnSignal
0,2004.0,54.0,109450.0,22081.0,1.0,83779400.0,1.0,0.0,0.0,3.0,2.0,17.0,40.236396,-83.366465,276.0,275.0,11.380001,0.0,0.0,1000.0,255.0,255.0,41.219460,-12.822062,0.0,0.0,40.236264,-83.366918,4.32000,267.22000,6.277778,0.0,-0.970000,0.8282,0.0
1,2004.0,55.0,9800.0,24156.0,1.0,86338800.0,1.0,0.0,0.0,3.0,2.0,16.0,40.236347,-83.365770,276.0,275.0,10.820000,0.0,0.0,1000.0,255.0,255.0,7.493227,-3.341818,0.0,0.0,40.236408,-83.365727,194.19000,266.22000,3.941667,0.0,18.059999,0.7894,2.0
2,2008.0,61.0,222650.0,11943.0,1.0,44211400.0,1.0,0.0,0.0,2.0,2.0,16.0,40.275613,-83.535510,55.0,300.0,22.760002,0.0,0.0,1000.0,255.0,255.0,102.194890,-22.708305,0.0,0.0,40.276089,-83.534481,359.97000,299.64001,0.000000,1.0,0.000000,0.0716,0.0
3,2147.0,156.0,136900.0,20935.0,1.0,46186700.0,1.0,0.0,0.0,2.0,2.0,16.0,40.275897,-83.534973,55.0,300.0,25.000000,0.0,0.0,1000.0,255.0,255.0,54.482815,-24.799370,0.0,0.0,40.276081,-83.534470,3.00000,299.67001,0.000000,1.0,0.240000,0.0134,2.0
4,2331.0,20.0,194450.0,13652.0,1.0,47461400.0,1.0,0.0,0.0,2.0,2.0,16.0,40.275970,-83.534832,55.0,300.0,14.160000,0.0,0.0,1000.0,255.0,255.0,35.005138,-13.796993,0.0,0.0,40.276081,-83.534464,358.73999,298.64999,0.000000,1.0,0.020000,-0.0060,2.0


In [54]:
# calculate reaction time using host table . max look for 60s
max_look_time = 60 * 100
reactiveness = pd.DataFrame(columns=['Device','Trip','ReactionTime','Time'])
for i,row in e_.iterrows():
    rt = list(host[(host['Device']==row['Device']) & (host['Trip']==row['Trip']) & (host['Time']>row['Time']) & (host['Time']<(row['Time']+max_look_time)) & (host['BrakeStatus']==1)]['Time'])
    if len(rt)>0:
        rt=(rt[0]-row['Time'])/100
    else:
        rt=-1
    reactiveness.loc[i]=[row['Device'],row['Trip'],rt,row['Time']]

In [55]:
reactiveness

,Device,Trip,ReactionTime,Time
0,2004.0,54.0,6.0,109450.0
1,2004.0,55.0,10.0,9800.0
2,2008.0,61.0,0.5,222650.0
3,2147.0,156.0,0.5,136900.0
4,2331.0,20.0,0.5,194450.0
5,2331.0,142.0,0.5,221800.0
6,2331.0,189.0,0.5,127600.0
7,2331.0,298.0,0.5,133700.0
8,2331.0,323.0,0.5,241500.0
9,2331.0,380.0,0.5,136600.0


In [57]:
evtwarn['EventAppId'].value_counts()

11    796
3     521
2     482
12    414
10    152
1      58
0      26
9      12
Name: EventAppId, dtype: int64

In [58]:
evt_ = evtwarn[(evtwarn['EventAppId'] == 0)]
ignore_interval = 2.5 * 100
e_  = pd.DataFrame(columns=evt_.columns)
i_ = 0
for i, row in evt_.iterrows():
    if i_==0:
        e_.loc[i_] = row
        i_+=1
    else:
        if ((i-1 in evt_.index) and (evt_.loc[i]['Trip']==evt_.loc[i-1]['Trip']) \
        and (evt_.loc[i]['Device']==evt_.loc[i-1]['Device']) and (evt_.loc[i]['Time']-evt_.loc[i-1]['Time']<=ignore_interval)):
            pass
        else:
            e_.loc[i_] = row
            i_+=1
max_look_time = 60 * 100
reactiveness = pd.DataFrame(columns=['Device','Trip','ReactionTime','Time'])
for i,row in e_.iterrows():
    rt = list(host[(host['Device']==row['Device']) & (host['Trip']==row['Trip']) & (host['Time']>row['Time']) & (host['Time']<(row['Time']+max_look_time)) & (host['BrakeStatus']==1)]['Time'])
    if len(rt)>0:
        rt=(rt[0]-row['Time'])/100
    else:
        rt=-1
    reactiveness.loc[i]=[row['Device'],row['Trip'],rt,row['Time']]
reactiveness

,Device,Trip,ReactionTime,Time
0,2004.0,119.0,2.0,129650.0
1,2494.0,476.0,4.5,278050.0
2,2941.0,182.0,2.0,93900.0
3,2941.0,182.0,-1.0,157200.0
4,2941.0,182.0,-1.0,158500.0
5,2941.0,182.0,-1.0,159250.0
6,2941.0,182.0,-1.0,159750.0


In [59]:
evt_ = evtwarn[(evtwarn['EventAppId'] == 1)]
ignore_interval = 2.5 * 100
e_  = pd.DataFrame(columns=evt_.columns)
i_ = 0
for i, row in evt_.iterrows():
    if i_==0:
        e_.loc[i_] = row
        i_+=1
    else:
        if ((i-1 in evt_.index) and (evt_.loc[i]['Trip']==evt_.loc[i-1]['Trip']) \
        and (evt_.loc[i]['Device']==evt_.loc[i-1]['Device']) and (evt_.loc[i]['Time']-evt_.loc[i-1]['Time']<=ignore_interval)):
            pass
        else:
            e_.loc[i_] = row
            i_+=1
max_look_time = 60 * 100
reactiveness = pd.DataFrame(columns=['Device','Trip','ReactionTime','Time'])
for i,row in e_.iterrows():
    rt = list(host[(host['Device']==row['Device']) & (host['Trip']==row['Trip']) & (host['Time']>row['Time']) & (host['Time']<(row['Time']+max_look_time)) & (host['BrakeStatus']==1)]['Time'])
    if len(rt)>0:
        rt=(rt[0]-row['Time'])/100
    else:
        rt=-1
    reactiveness.loc[i]=[row['Device'],row['Trip'],rt,row['Time']]
reactiveness

,Device,Trip,ReactionTime,Time
0,2008.0,26.0,-1.0,92500.0
1,2017.0,301.0,-1.0,59350.0
2,2348.0,35.0,-1.0,82500.0
3,2496.0,94.0,3.5,27000.0
4,2496.0,95.0,26.0,206300.0
5,2496.0,99.0,-1.0,37750.0
6,2496.0,99.0,-1.0,37950.0
7,2496.0,100.0,27.0,13250.0
8,2496.0,100.0,47.5,29150.0
9,2496.0,100.0,-1.0,111250.0


In [60]:
evt_ = evtwarn[(evtwarn['EventAppId'] == 3)]
ignore_interval = 2.5 * 100
e_  = pd.DataFrame(columns=evt_.columns)
i_ = 0
for i, row in evt_.iterrows():
    if i_==0:
        e_.loc[i_] = row
        i_+=1
    else:
        if ((i-1 in evt_.index) and (evt_.loc[i]['Trip']==evt_.loc[i-1]['Trip']) \
        and (evt_.loc[i]['Device']==evt_.loc[i-1]['Device']) and (evt_.loc[i]['Time']-evt_.loc[i-1]['Time']<=ignore_interval)):
            pass
        else:
            e_.loc[i_] = row
            i_+=1
max_look_time = 60 * 100
reactiveness = pd.DataFrame(columns=['Device','Trip','ReactionTime','Time'])
for i,row in e_.iterrows():
    rt = list(host[(host['Device']==row['Device']) & (host['Trip']==row['Trip']) & (host['Time']>row['Time']) & (host['Time']<(row['Time']+max_look_time)) & (host['BrakeStatus']==1)]['Time'])
    if len(rt)>0:
        rt=(rt[0]-row['Time'])/100
    else:
        rt=-1
    reactiveness.loc[i]=[row['Device'],row['Trip'],rt,row['Time']]
reactiveness

,Device,Trip,ReactionTime,Time
0,2004.0,12.0,-1.0,48150.0
1,2008.0,8.0,-1.0,119650.0
2,2008.0,20.0,-1.0,99150.0
3,2008.0,25.0,-1.0,108350.0
4,2008.0,26.0,-1.0,91450.0
5,2008.0,61.0,42.0,130750.0
6,2017.0,81.0,-1.0,219550.0
7,2017.0,301.0,33.0,101400.0
8,2147.0,181.0,-1.0,67250.0
9,2147.0,198.0,-1.0,62500.0


In [61]:
evt_ = evtwarn[(evtwarn['EventAppId'] == 9)]
ignore_interval = 2.5 * 100
e_  = pd.DataFrame(columns=evt_.columns)
i_ = 0
for i, row in evt_.iterrows():
    if i_==0:
        e_.loc[i_] = row
        i_+=1
    else:
        if ((i-1 in evt_.index) and (evt_.loc[i]['Trip']==evt_.loc[i-1]['Trip']) \
        and (evt_.loc[i]['Device']==evt_.loc[i-1]['Device']) and (evt_.loc[i]['Time']-evt_.loc[i-1]['Time']<=ignore_interval)):
            pass
        else:
            e_.loc[i_] = row
            i_+=1
max_look_time = 60 * 100
reactiveness = pd.DataFrame(columns=['Device','Trip','ReactionTime','Time'])
for i,row in e_.iterrows():
    rt = list(host[(host['Device']==row['Device']) & (host['Trip']==row['Trip']) & (host['Time']>row['Time']) & (host['Time']<(row['Time']+max_look_time)) & (host['BrakeStatus']==1)]['Time'])
    if len(rt)>0:
        rt=(rt[0]-row['Time'])/100
    else:
        rt=-1
    reactiveness.loc[i]=[row['Device'],row['Trip'],rt,row['Time']]
reactiveness

,Device,Trip,ReactionTime,Time
0,2584.0,43.0,0.5,30100.0
1,2584.0,402.0,1.0,18600.0
2,2584.0,402.0,0.5,18950.0


In [62]:
evt_ = evtwarn[(evtwarn['EventAppId'] == 10)]
ignore_interval = 2.5 * 100
e_  = pd.DataFrame(columns=evt_.columns)
i_ = 0
for i, row in evt_.iterrows():
    if i_==0:
        e_.loc[i_] = row
        i_+=1
    else:
        if ((i-1 in evt_.index) and (evt_.loc[i]['Trip']==evt_.loc[i-1]['Trip']) \
        and (evt_.loc[i]['Device']==evt_.loc[i-1]['Device']) and (evt_.loc[i]['Time']-evt_.loc[i-1]['Time']<=ignore_interval)):
            pass
        else:
            e_.loc[i_] = row
            i_+=1
max_look_time = 60 * 100
reactiveness = pd.DataFrame(columns=['Device','Trip','ReactionTime','Time'])
for i,row in e_.iterrows():
    rt = list(host[(host['Device']==row['Device']) & (host['Trip']==row['Trip']) & (host['Time']>row['Time']) & (host['Time']<(row['Time']+max_look_time)) & (host['BrakeStatus']==1)]['Time'])
    if len(rt)>0:
        rt=(rt[0]-row['Time'])/100
    else:
        rt=-1
    reactiveness.loc[i]=[row['Device'],row['Trip'],rt,row['Time']]
reactiveness

,Device,Trip,ReactionTime,Time
0,2107.0,164.0,0.5,249500.0
1,2107.0,301.0,2.5,31700.0
2,2331.0,215.0,10.5,118900.0
3,2348.0,35.0,0.5,316400.0
4,2348.0,35.0,4.0,317200.0
5,2348.0,35.0,9.5,325250.0
6,2348.0,35.0,0.5,328350.0
7,2348.0,35.0,11.5,328700.0
8,2348.0,35.0,0.5,359700.0
9,2348.0,35.0,9.5,360200.0


In [63]:
evt_ = evtwarn[(evtwarn['EventAppId'] == 11)]
ignore_interval = 2.5 * 100
e_  = pd.DataFrame(columns=evt_.columns)
i_ = 0
for i, row in evt_.iterrows():
    if i_==0:
        e_.loc[i_] = row
        i_+=1
    else:
        if ((i-1 in evt_.index) and (evt_.loc[i]['Trip']==evt_.loc[i-1]['Trip']) \
        and (evt_.loc[i]['Device']==evt_.loc[i-1]['Device']) and (evt_.loc[i]['Time']-evt_.loc[i-1]['Time']<=ignore_interval)):
            pass
        else:
            e_.loc[i_] = row
            i_+=1
max_look_time = 60 * 100
reactiveness = pd.DataFrame(columns=['Device','Trip','ReactionTime','Time'])
for i,row in e_.iterrows():
    rt = list(host[(host['Device']==row['Device']) & (host['Trip']==row['Trip']) & (host['Time']>row['Time']) & (host['Time']<(row['Time']+max_look_time)) & (host['BrakeStatus']==1)]['Time'])
    if len(rt)>0:
        rt=(rt[0]-row['Time'])/100
    else:
        rt=-1
    reactiveness.loc[i]=[row['Device'],row['Trip'],rt,row['Time']]
reactiveness

,Device,Trip,ReactionTime,Time
0,2004.0,55.0,0.5,11300.0
1,2008.0,5.0,9.0,9150.0
2,2233.0,12.0,1.5,107750.0
3,2494.0,398.0,0.5,192900.0
4,2496.0,315.0,0.5,17400.0
5,2496.0,315.0,0.5,17450.0
6,2496.0,334.0,1.5,95500.0
7,2496.0,422.0,14.0,183200.0
8,2496.0,442.0,15.0,5450.0
9,2496.0,442.0,7.0,6250.0


In [64]:
evt_ = evtwarn[(evtwarn['EventAppId'] == 12)]
ignore_interval = 2.5 * 100
e_  = pd.DataFrame(columns=evt_.columns)
i_ = 0
for i, row in evt_.iterrows():
    if i_==0:
        e_.loc[i_] = row
        i_+=1
    else:
        if ((i-1 in evt_.index) and (evt_.loc[i]['Trip']==evt_.loc[i-1]['Trip']) \
        and (evt_.loc[i]['Device']==evt_.loc[i-1]['Device']) and (evt_.loc[i]['Time']-evt_.loc[i-1]['Time']<=ignore_interval)):
            pass
        else:
            e_.loc[i_] = row
            i_+=1
max_look_time = 60 * 100
reactiveness = pd.DataFrame(columns=['Device','Trip','ReactionTime','Time'])
for i,row in e_.iterrows():
    rt = list(host[(host['Device']==row['Device']) & (host['Trip']==row['Trip']) & (host['Time']>row['Time']) & (host['Time']<(row['Time']+max_look_time)) & (host['BrakeStatus']==1)]['Time'])
    if len(rt)>0:
        rt=(rt[0]-row['Time'])/100
    else:
        rt=-1
    reactiveness.loc[i]=[row['Device'],row['Trip'],rt,row['Time']]
reactiveness

,Device,Trip,ReactionTime,Time
0,2004.0,55.0,1.5,10650.0
1,2496.0,100.0,14.0,96000.0
2,2496.0,422.0,3.0,204150.0
3,2496.0,422.0,3.0,219950.0
4,2496.0,422.0,2.5,234200.0
5,2496.0,422.0,4.0,249700.0
6,2496.0,422.0,2.0,267850.0
7,2496.0,422.0,2.0,286700.0
8,2496.0,444.0,-1.0,176550.0
9,2496.0,444.0,26.0,491250.0
